# Releases Visualisation

In [1]:
import requests
import pandas as pd
import re
import json
from IPython.core.display import display, HTML
from modules.Network import *
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from
https://wmwaredata.s3.us-east-2.amazonaws.com/releases.json

In [3]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/releases.json'
r = requests.get(url, allow_redirects=True)
open('data/releases.json', 'wb').write(r.content)

40979

In [4]:
df = pd.read_json('data/releases.json')

descriptions = []
for i in range(len(df)):
    content = re.sub('<.*?>', '', df['release_name'].iloc[i])
    des = wrap_by_word(content, n=5)
    des = wrap_by_char(des, n=45)
    descriptions.append(des)
    
df['description'] = descriptions
df = df.drop('release_name', axis=1)
df = df.rename(columns={'repo_repo': 'repo_name'})
df = df[['repo_name', 'repo_url', 'release_tag', 'release_date', 'description']]
df

,repo_name,repo_url,release_tag,release_date,description
0,GW-proxy,https://github.com/k8-proxy/GW-proxy,v0.5.1,2021-01-08T16:40:48Z,
1,k8-reverse-proxy,https://github.com/k8-proxy/k8-reverse-proxy,v0.1.0,2020-10-26T13:40:49Z,
2,k8-test-data,https://github.com/k8-proxy/k8-test-data,k-8-test-data-v0.4,2020-10-29T22:34:08Z,* Processed 18k gov uk\nsite files via test da...
3,k8-test-data,https://github.com/k8-proxy/k8-test-data,k-8-test-data-v0.3,2020-10-14T10:27:07Z,### Features * Integration of\nDistribution Ap...
4,k8-test-data,https://github.com/k8-proxy/k8-test-data,k-8-test-data-v0.2,2020-10-06T16:00:25Z,## Features * Web scraping\nfor below sites ha...
...,...,...,...,...,...
74,GW-Releases,https://github.com/k8-proxy/GW-Releases,v0.5.1,2021-01-12T14:17:23Z,Release V5.1.0 contains the following\ntested ...
75,k8-rebuild-folder-to-folder,https://github.com/k8-proxy/k8-rebuild-folder-...,v0.1.1,2021-02-03T14:49:50Z,Fixes on v0.1.1: 1.- Some\nzip files were not ...
76,k8-rebuild-folder-to-folder,https://github.com/k8-proxy/k8-rebuild-folder-...,v0.1.0,2021-01-26T09:24:51Z,The initial docker version of\nk8-rebuild fold...
77,icap-client,https://github.com/k8-proxy/icap-client,v0.2.1,2021-02-17T19:33:50Z,


## Repo's Releases

In [5]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()
network.groups = ['Repo', 'Tag', 'Date', 'Description']
network.groupSettings = group_settings_R

for repo in set(list(df['repo_name'])):
    
    # REPO
    network.add_node(label=repo, group='Repo')
    df1 = df[df['repo_name'] == repo].reset_index()
    
    for i in range(len(df1)):
        # TAG
        tag = 'Tag: ' + df1['release_tag'].iloc[i] 
        network.add_node(label=tag, group='Tag', repeat_nodes=True)
        network.add_edge(network.nodesIds[repo][0], network.nodesIds[tag][-1])
        df2 = df1[df1['release_tag'] == tag].reset_index()
        
        # DATE
        d = df1['release_date'].iloc[i]
        
        if d is not None:
            d = d.split('T')
            date = 'Date: '+ d[0] + '\nTime: ' + d[1][:-1]
        else:
            date = ''
        
        network.add_node(label=date, group='Date', repeat_nodes=True)
        network.add_edge(network.nodesIds[tag][-1], network.nodesIds[date][-1])
        
        # DESCRIPTION
        des = df1['description'].iloc[i]
        if des != '':
            network.add_node(label=des, group='Description', repeat_nodes=True)
            network.add_edge(network.nodesIds[date][-1], network.nodesIds[des][-1])
    
network.save_to_json('data/releases_nodes_edges.json')

In [6]:
%%html
<div id="mynetwork"></div>

In [7]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/releases_nodes_edges.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1000px',
      height: '800px',
      locale: 'en',
      physics: true,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
      }
    };    
    
    var container = document.getElementById("mynetwork");
});

<IPython.core.display.Javascript object>